In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import joblib
import os

FIXED_SEASONS = ['Autumn', 'Spring', 'Summer', 'Winter']
FIXED_CATEGORIES = [
    'apartman', 'családi ház', 'gyár', 'hivatal', 'ikerház', 
    'irodaház', 'iskola', 'kertes ház', 'kunyhó', 'könyvtár', 
    'panel lakás', 'sorház', 'tanya'
]
NUMERICAL_COLS = [
    'production', 'air_temp', 'cloud_opacity', 'dhi', 'dni', 'ghi', 'gti',
    'clearsky_dhi', 'clearsky_dni', 'clearsky_ghi', 'clearsky_gti',
    'snow_soiling_rooftop', 'snow_soiling_ground',
    'number_of_panels', 'panel_area_m2' 
]
FINAL_FEATURE_ORDER = NUMERICAL_COLS + \
                      [f'season_{s}' for s in FIXED_SEASONS] + \
                      [f'category_{c}' for c in FIXED_CATEGORIES]


class SolarGRU(nn.Module):
    def __init__(self, input_size=32, hidden_size=128, num_layers=2, output_size=1):
        super(SolarGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=0.1)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        device = x.device
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

class SolarPredictor:
    def __init__(self, model_path='Communities_GRU.pth', scaler_x_path='GRU_scaler_X.save', scaler_y_path='GRU_scaler_y.save'):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # 1. Betöltjük a skálázókat
        self.scaler_X = joblib.load(scaler_x_path)
        self.scaler_y = joblib.load(scaler_y_path)

        # 2. Betöltjük a modellt
        self.model = SolarGRU(input_size=len(FINAL_FEATURE_ORDER)).to(self.device)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval()
        print("✅ Modell sikeresen betöltve.")

    def preprocess(self, df_input):
        """
        Átalakítja a nyers DataFrame-et a modell által várt formátumra (31 feature).
        """
        df = df_input.copy()
        
        # Fizikai pótlás
        if 'panel_area_m2' not in df.columns:
            if 'number_of_panels' in df.columns:
                df['panel_area_m2'] = df['number_of_panels'] * 1.7
            else:
                df['panel_area_m2'] = 0.0
        
        # Kategóriák kezelése
        df['season'] = pd.Categorical(df['season'], categories=FIXED_SEASONS)
        df['category'] = pd.Categorical(df['category'], categories=FIXED_CATEGORIES)
        
        # One-Hot Encoding
        df = pd.get_dummies(df, columns=['season', 'category'], dummy_na=False, dtype='float32')
        
        # Hiányzó oszlopok pótlása 0-val (Fix 31 feature garantálása)
        for col in FINAL_FEATURE_ORDER:
            if col not in df.columns:
                df[col] = 0.0
                
        # Sorrendezés és konvertálás
        X = df[FINAL_FEATURE_ORDER].values.astype('float32')
        
        # Skálázás
        X_scaled = self.scaler_X.transform(X)
        return X_scaled

    def predict(self, df_history):
        """
        Fő függvény.
        Bemenet: Pandas DataFrame (legalább 96 soros előzmény)
        Kimenet: Float (a becsült termelés kWh-ban)
        """
        
        # utolsó 96 órát vesszük figyelembe
        df_recent = df_history.iloc[-96:].copy()
        
        # Előkészítés
        X_scaled = self.preprocess(df_recent)
        
        input_tensor = torch.tensor(X_scaled, dtype=torch.float32).unsqueeze(0).to(self.device)
        
        # Becslés
        with torch.no_grad():
            prediction_norm = self.model(input_tensor).cpu().numpy()
            
        # Visszaskálázás
        prediction_real = self.scaler_y.inverse_transform(prediction_norm)
        
        # Fizikai korlát (negatív nem lehet)
        result = max(0.0, float(prediction_real[0][0]))
        
        return result

In [ ]:

import pandas as pd
import numpy as np
import joblib
import os

# --- HASZNÁLATI PÉLDA (Ha közvetlenül futtatják ezt a fájlt) ---
if __name__ == "__main__":
    #(elmúlt 96 óra szimulációja)
    
    mock_data = []
    
    start_offset = 13 
    
    for i in range(96):
        current_hour = (i + start_offset) % 24

        production = np.random.uniform(0, 1000)

        # Napszak logika: Reggel 6 és este 8 között van nap
        is_daytime = 6 <= current_hour <= 20
        # Csúcsidő: 10 és 15 óra között
        is_peak = 10 <= current_hour <= 15
        
        if is_peak:
            # Erős napsütés délben
            gti = np.random.uniform(800, 950)
            dni = np.random.uniform(750, 900)
            ghi = np.random.uniform(800, 900)
            temp = 30.0
        elif is_daytime:
            # Gyengébb nap reggel/este
            gti = np.random.uniform(100, 400)
            dni = np.random.uniform(50, 300)
            ghi = np.random.uniform(100, 400)
            temp = 20.0
        else:
            # Éjszaka
            gti = 0; dni = 0; ghi = 0; temp = 15.0

        mock_data.append({
            'production': production,
            'number_of_panels': 20,
            'panel_area_m2': 34.0, # 20 * 1.7
            'category': 'családi ház',
            'air_temp': temp,
            'clearsky_dhi': 100 if is_daytime else 0,
            'clearsky_dni': dni + 50 if is_daytime else 0,
            'clearsky_ghi': ghi + 50 if is_daytime else 0,
            'clearsky_gti': gti + 50 if is_daytime else 0,
            'dhi': 100 if is_daytime else 0,
            'dni': dni,
            'ghi': ghi,
            'gti': gti,
            'cloud_opacity': 0, # Tiszta ég
            'season': 'Summer',
            'snow_soiling_ground': 0,
            'snow_soiling_rooftop': 0
        })

    df_test = pd.DataFrame(mock_data)

    df_test.to_csv('Communities_GRU_test.txt', mode='a', header=False, index=False, sep=';', encoding='utf-8')
    df_test = pd.DataFrame(mock_data)
    
    #Ellenőrzés: Kiírjuk az utolsó sor idejét és sugárzását
    last_row = df_test.iloc[-1]
    last_hour = (95 + start_offset) % 24
    print(f"Utolsó szimulált óra: {last_hour}:00")
    print(f"Utolsó GTI sugárzás: {last_row['gti']:.2f} W/m2") # Ennek magasnak kell lennie!
    
    # 2. Predictor inicializálása és jóslás
    try:
        predictor = SolarPredictor()
        eredmeny = predictor.predict(df_test)
        print(f"\n✅ BECSÜLT TERMELÉS (Nyári dél): {eredmeny:.4f} kWh")
    except Exception as e:
        print(f"\n⚠️ Hiba: {e}")

Utolsó szimulált óra: 12:00
Utolsó GTI sugárzás: 873.47 W/m2

⚠️ Hiba: name 'SolarPredictor' is not defined
